In [ ]:
#Image resizing
from PIL import Image

# Open the .tif image
image = Image.open("Ld2.tif")

# Resize the image to 512x512 pixels
resized_image = image.resize((512, 512))

# Save the resized image
resized_image.save("Ld2.tif")


In [ ]:
#Converting lum .tif images to .jpg
from PIL import Image

# Open the .tif image
tif_image = Image.open("lum_L_d2.tif")

# Convert the image to RGB mode (required for JPEG format)
rgb_image = tif_image.convert("RGB")

# Save the image in JPEG format
rgb_image.save("lum_L_d2.jpg", "JPEG")


In [ ]:
#Brightening well plate images
from PIL import Image, ImageEnhance
import cv2
import numpy as np
from matplotlib import pyplot as plt


# Open a TIFF image file
image = Image.open('Rd2.TIF')

# Convert the image to RGB
image = image.convert('RGB')

# Enhance the image contrast and brightness
enhancer_contrast = ImageEnhance.Contrast(image)
contrast_image = enhancer_contrast.enhance(8.0)  # Increase contrast

enhancer_brightness = ImageEnhance.Brightness(contrast_image)
bright_image = enhancer_brightness.enhance(1.0)  # Increase brightness

# Convert the PIL Image to a NumPy array
bright_image_np = np.array(bright_image)

# Convert the NumPy array to a format OpenCV understands
bright_image_cv = cv2.cvtColor(bright_image_np, cv2.COLOR_RGB2BGR)

# Save the brightened image using OpenCV
cv2.imwrite("Rd2.jpg", bright_image_cv)

In [ ]:
#Cropping and joining lum and well plate images
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper and Merger")

        # Initialize image variables with explicit file names
        self.image1 = Image.open('Ld2.jpg')
        self.image2 = Image.open('Rd2.jpg')
        self.corresponding_image1 = Image.open('lum_L_d2.jpg')
        self.corresponding_image2 = Image.open('lum_R_d2.jpg')
        self.cropped_image1 = None
        self.cropped_image2 = None
        self.cropped_corresponding_image1 = None
        self.cropped_corresponding_image2 = None

        # Set up the main frame
        self.main_frame = tk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Create GUI components
        self.create_widgets()

    def create_widgets(self):
        # Buttons frame
        button_frame = tk.Frame(self.main_frame)
        button_frame.pack(side=tk.TOP, fill=tk.X)

        # Crop images button
        self.crop_button = tk.Button(button_frame, text="Crop Images", command=self.crop_images)
        self.crop_button.pack(side=tk.LEFT, padx=5, pady=5)

        # Join images button
        self.join_button = tk.Button(button_frame, text="Join Images", command=self.join_images)
        self.join_button.pack(side=tk.LEFT, padx=5, pady=5)

        # Canvas to display images
        self.canvas_frame = tk.Frame(self.main_frame)
        self.canvas_frame.pack(fill=tk.BOTH, expand=True)
        
        self.canvas = tk.Canvas(self.canvas_frame, bg='gray')
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.v_scroll = tk.Scrollbar(self.canvas_frame, orient=tk.VERTICAL, command=self.canvas.yview)
        self.v_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.h_scroll = tk.Scrollbar(self.main_frame, orient=tk.HORIZONTAL, command=self.canvas.xview)
        self.h_scroll.pack(side=tk.BOTTOM, fill=tk.X)
        
        self.canvas.config(yscrollcommand=self.v_scroll.set, xscrollcommand=self.h_scroll.set)

        # Display the initial images
        self.display_images()

    def display_images(self):
        self.canvas.delete("all")

        # Calculate scaling factor
        max_width = 800  # Maximum width for display
        total_width = self.image1.width + self.image2.width
        scale_factor = min(max_width / total_width, 1)

        # Resize images for display
        display_image1 = self.image1.copy()
        display_image2 = self.image2.copy()
        if scale_factor < 1:
            new_size1 = (int(self.image1.width * scale_factor), int(self.image1.height * scale_factor))
            new_size2 = (int(self.image2.width * scale_factor), int(self.image2.height * scale_factor))
            if new_size1[0] > 0 and new_size1[1] > 0:
                display_image1 = display_image1.resize(new_size1, Image.LANCZOS)
            if new_size2[0] > 0 and new_size2[1] > 0:
                display_image2 = display_image2.resize(new_size2, Image.LANCZOS)

        # Calculate canvas size
        total_width = display_image1.width + display_image2.width
        max_height = max(display_image1.height, display_image2.height)

        # Configure canvas and scrollregion
        self.canvas.config(scrollregion=(0, 0, total_width, max_height))

        # Convert images to PhotoImage
        self.tk_image1 = ImageTk.PhotoImage(display_image1)
        self.tk_image2 = ImageTk.PhotoImage(display_image2)

        # Display images in the canvas
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image1)
        self.canvas.create_image(display_image1.width, 0, anchor=tk.NW, image=self.tk_image2)

    def crop_images(self):
        # Ask user to crop the images
        self.cropped_image1, crop_box1 = self.ask_crop(self.image1, "Crop Image 1")
        if self.cropped_image1 is None:
            return  # User canceled cropping
        self.cropped_image2, crop_box2 = self.ask_crop(self.image2, "Crop Image 2")
        if self.cropped_image2 is None:
            return  # User canceled cropping

        # Crop corresponding images using the same crop boxes
        self.cropped_corresponding_image1 = self.corresponding_image1.crop(crop_box1)
        self.cropped_corresponding_image2 = self.corresponding_image2.crop(crop_box2)

        # Update main images with cropped versions
        self.image1 = self.cropped_image1
        self.image2 = self.cropped_image2
        self.display_images()

    def ask_crop(self, image, title):
        crop_window = tk.Toplevel(self.root)
        crop_window.title(title)

        # Create a frame for the canvas and scrollbars
        frame = tk.Frame(crop_window)
        frame.pack(fill=tk.BOTH, expand=True)

        # Create a canvas with scrollbars
        canvas = tk.Canvas(frame, bg='gray')
        v_scroll = tk.Scrollbar(frame, orient=tk.VERTICAL, command=canvas.yview)
        h_scroll = tk.Scrollbar(crop_window, orient=tk.HORIZONTAL, command=canvas.xview)
        canvas.configure(yscrollcommand=v_scroll.set, xscrollcommand=h_scroll.set)
        canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        v_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        h_scroll.pack(side=tk.BOTTOM, fill=tk.X)

        # Calculate scaling factor
        max_size = 800  # Maximum size for display
        scale_factor = min(max_size / max(image.width, image.height), 1)

        # Resize image for display if necessary
        if scale_factor < 1:
            display_image = image.copy()
            new_size = (int(image.width * scale_factor), int(image.height * scale_factor))
            display_image = display_image.resize(new_size, Image.LANCZOS)
        else:
            display_image = image

        # Update canvas size and scrollregion
        canvas.config(scrollregion=(0, 0, display_image.width, display_image.height))

        # Convert to PhotoImage for Tkinter
        self.tk_crop_image = ImageTk.PhotoImage(display_image)
        canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_crop_image)

        # Variables to store cropping coordinates
        self.crop_start = None
        self.crop_end = None
        self.cropped_image = None
        self.rect = None

        def on_button_press(event):
            canvas_x = canvas.canvasx(event.x)
            canvas_y = canvas.canvasy(event.y)
            self.crop_start = (canvas_x, canvas_y)
            if self.rect:
                canvas.delete(self.rect)
            self.rect = canvas.create_rectangle(canvas_x, canvas_y, canvas_x, canvas_y, outline='red')

        def on_motion(event):
            if self.crop_start:
                canvas_x = canvas.canvasx(event.x)
                canvas_y = canvas.canvasy(event.y)
                canvas.coords(self.rect, self.crop_start[0], self.crop_start[1], canvas_x, canvas_y)

        def on_button_release(event):
            canvas_x = canvas.canvasx(event.x)
            canvas_y = canvas.canvasy(event.y)
            self.crop_end = (canvas_x, canvas_y)
            if self.crop_start and self.crop_end:
                self.crop_and_display(canvas, image, scale_factor)
                self.confirm_cropping(crop_window)

        canvas.bind("<ButtonPress-1>", on_button_press)
        canvas.bind("<B1-Motion>", on_motion)
        canvas.bind("<ButtonRelease-1>", on_button_release)

        crop_window.wait_window()

        return self.cropped_image, self.crop_box

    def crop_and_display(self, canvas, image, scale_factor):
        if not self.crop_start or not self.crop_end:
            return

        # Convert crop coordinates back to original image size
        x1, y1 = int(self.crop_start[0] / scale_factor), int(self.crop_start[1] / scale_factor)
        x2, y2 = int(self.crop_end[0] / scale_factor), int(self.crop_end[1] / scale_factor)
        if x1 > x2:
            x1, x2 = x2, x1
        if y1 > y2:
            y1, y2 = y2, y1

        self.crop_box = (x1, y1, x2, y2)
        cropped = image.crop(self.crop_box)
        self.cropped_image = cropped

        # Display cropped image
        display_cropped = cropped.copy()
        display_cropped.thumbnail((400, 400))
        self.tk_cropped = ImageTk.PhotoImage(display_cropped)
        canvas.delete("all")
        canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_cropped)

    def confirm_cropping(self, crop_window):
        response = messagebox.askyesno("Confirm Cropping", "Do you want to keep this cropped image?")
        if not response:
            self.cropped_image = None
            self.crop_box = None
        crop_window.destroy()

    def join_images(self):
        if self.cropped_image1 is None or self.cropped_image2 is None:
            messagebox.showwarning("Warning", "Please crop both images first.")
            return

        # Join normal images side by side
        joined_normal = self.join_image_pair(self.cropped_image1, self.cropped_image2)

        # Join corresponding images side by side
        joined_corresponding = self.join_image_pair(self.cropped_corresponding_image1, self.cropped_corresponding_image2)

        # Update the display with the joined normal images
        self.image1 = joined_normal
        self.image2 = Image.new("RGB", (1, 1))  # Dummy image
        self.display_images()

        # Ask user to save the final joined images
        self.save_image(joined_normal, "normal")
        self.save_image(joined_corresponding, "corresponding")

    def join_image_pair(self, img1, img2):
        total_width = img1.width + img2.width
        max_height = max(img1.height, img2.height)
        new_image = Image.new("RGB", (total_width, max_height))

        vertical_offset = (max_height - img2.height) // 2

        new_image.paste(img1, (0, 0))
        new_image.paste(img2, (img1.width, vertical_offset))

        return new_image

    def save_image(self, image, image_type):
        file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                               filetypes=[("PNG files", "*.png"),
                                                          ("JPEG files", "*.jpg"),
                                                          ("All files", "*.*")],
                                               initialfile=f"joined_{image_type}_image")
        if file_path:
            image.save(file_path)
            messagebox.showinfo("Save Image", f"{image_type.capitalize()} image saved as {file_path}")

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageCropper(root)
    root.mainloop()

In [ ]:
#Image analysis and drug assignment
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tkinter as tk
from tkinter import messagebox, simpledialog

# Load the original image and the luminescence image
original_image = cv2.imread("d2.png")
lum_image = cv2.imread("lum_d2.png")

# Ensure both images are successfully loaded
if original_image is None or lum_image is None:
    print("One of the images could not be loaded. Please check the file paths.")
    exit()

# Ensure both images are the same size
if original_image.shape != lum_image.shape:
    lum_image = cv2.resize(lum_image, (original_image.shape[1], original_image.shape[0]))

# Convert the original image to grayscale
gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur and adaptive thresholding
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
adaptive_thresh = cv2.adaptiveThreshold(
    blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
)

# Perform morphological operations
kernel = np.ones((3, 3), np.uint8)
morph = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_CLOSE, kernel)
morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

# Find contours
contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours by area and aspect ratio
min_area = 50
max_area = 1000
filtered_contours = []
for cnt in contours:
    area = cv2.contourArea(cnt)
    if min_area < area < max_area:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = float(w) / h
        if 0.5 < aspect_ratio < 2.0:
            filtered_contours.append(cnt)

# Function to get the bounding box center
def get_contour_center(cnt):
    x, y, w, h = cv2.boundingRect(cnt)
    return (x + w // 2, y + h // 2)

# Sort contours from top-left to bottom-right
filtered_contours.sort(key=lambda c: get_contour_center(c)[1] * 10000 + get_contour_center(c)[0])

# Define the grid size
grid_rows = 16
grid_cols = 24

# Find the bounding box that contains all contours
all_points = np.concatenate([cnt.reshape(-1, 2) for cnt in filtered_contours])
x_min, y_min = all_points.min(axis=0)
x_max, y_max = all_points.max(axis=0)

# Calculate grid cell size based on the bounding box
cell_width = (x_max - x_min) / grid_cols
cell_height = (y_max - y_min) / grid_rows

# Initialize DataFrame to store average values and drugs together
df = pd.DataFrame('', columns=[f'Col {i+1}' for i in range(grid_cols)],
                  index=[f'Row {i+1}' for i in range(grid_rows)])

# Draw contours and calculate average values
lum_image_with_contours = lum_image.copy()
original_image_with_contours = original_image.copy()

for i, cnt in enumerate(filtered_contours):
    # Fit a rotated rectangle to the contour
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    
    # Draw the rotated rectangle with thinner lines
    cv2.drawContours(lum_image_with_contours, [box], 0, (0, 255, 0), 1)
    cv2.drawContours(original_image_with_contours, [box], 0, (0, 255, 0), 1)
    
    # Create a mask for this contour
    mask = np.zeros(gray.shape, np.uint8)
    cv2.drawContours(mask, [cnt], 0, 255, -1)
    
    # Calculate the average pixel value within the contour
    mean_val = cv2.mean(cv2.cvtColor(lum_image, cv2.COLOR_BGR2GRAY), mask=mask)
    
    # Get contour center
    cX, cY = get_contour_center(cnt)
    
    # Calculate grid position
    col = int((cX - x_min) // cell_width)
    row = int((cY - y_min) // cell_height)
    
    # Ensure that we don't go out of grid bounds
    if 0 <= row < grid_rows and 0 <= col < grid_cols:
        df.iloc[row, col] = f"{mean_val[0]:.1f}"
        
        # Label each contour with its grid position
        label = f"{row + 1},{col + 1}"
        cv2.putText(lum_image_with_contours, label, (cX - 20, cY + 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        cv2.putText(original_image_with_contours, label, (cX - 20, cY + 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    print(f"Contour in grid ({row + 1},{col + 1}) average pixel value: {mean_val[0]}")

# Function to assign drugs
def assign_drug_to_row():
    row_number = simpledialog.askinteger("Input", "Enter row number (1-16):", parent=root)
    drug_name = simpledialog.askstring("Input", "Enter the drug name:", parent=root)
    if row_number is not None and drug_name:
        df.iloc[row_number-1, :] = df.iloc[row_number-1, :].apply(lambda x: f"{x} ({drug_name})")

def assign_drug_to_column():
    col_number = simpledialog.askinteger("Input", "Enter column number (1-24):", parent=root)
    drug_name = simpledialog.askstring("Input", "Enter the drug name:", parent=root)
    if col_number is not None and drug_name:
        df.iloc[:, col_number-1] = df.iloc[:, col_number-1].apply(lambda x: f"{x} ({drug_name})")

def assign_drug_to_range():
    row_start = simpledialog.askinteger("Input", "Enter starting row number (1-16):", parent=root)
    row_end = simpledialog.askinteger("Input", "Enter ending row number (1-16):", parent=root)
    col_start = simpledialog.askinteger("Input", "Enter starting column number (1-24):", parent=root)
    col_end = simpledialog.askinteger("Input", "Enter ending column number (1-24):", parent=root)
    drug_name = simpledialog.askstring("Input", "Enter the drug name:", parent=root)
    if row_start and row_end and col_start and col_end and drug_name:
        df.iloc[row_start-1:row_end, col_start-1:col_end] = df.iloc[row_start-1:row_end, col_start-1:col_end].applymap(lambda x: f"{x} ({drug_name})")

def assign_drug_to_well():
    row_number = simpledialog.askinteger("Input", "Enter row number (1-16):", parent=root)
    col_number = simpledialog.askinteger("Input", "Enter column number (1-24):", parent=root)
    drug_name = simpledialog.askstring("Input", "Enter the drug name:", parent=root)
    if row_number and col_number and drug_name:
        df.iloc[row_number-1, col_number-1] = f"{df.iloc[row_number-1, col_number-1]} ({drug_name})"

# Setup Tkinter GUI
root = tk.Tk()
root.title("Drug Assignment GUI")
root.geometry("400x300")

# Main menu for drug assignment options
assign_row_button = tk.Button(root, text="Assign Drug to Row", command=assign_drug_to_row)
assign_row_button.pack(pady=5)

assign_column_button = tk.Button(root, text="Assign Drug to Column", command=assign_drug_to_column)
assign_column_button.pack(pady=5)

assign_range_button = tk.Button(root, text="Assign Drug to Range", command=assign_drug_to_range)
assign_range_button.pack(pady=5)

assign_well_button = tk.Button(root, text="Assign Drug to Well", command=assign_drug_to_well)
assign_well_button.pack(pady=5)

# Save and exit button
finish_button = tk.Button(root, text="Finish and Generate Outputs", command=root.quit)
finish_button.pack(pady=20)

# Start the GUI
root.mainloop()

# Save the images with contours
cv2.imwrite('d2lumcont111.png', lum_image_with_contours)
cv2.imwrite('d2cont111.png', original_image_with_contours)

# Save the DataFrame to an Excel file
df.to_excel("luminescence_and_drugs.xlsx", index=True)

# Save the heatmap as an image
plt.figure(figsize=(12, 8))
sns.heatmap(df.applymap(lambda x: float(x.split()[0]) if x else np.nan), annot=True, fmt=".1f", cmap="coolwarm", cbar=True, linewidths=.5)
plt.title("Heatmap of Average Luminescence Values")
plt.savefig("heatmap.png")
plt.show()

# Display the original image, the luminescence image, and the images with contours side by side
plt.figure(figsize=(20, 10))
plt.subplot(141), plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)), plt.title('Original Image')
plt.subplot(142), plt.imshow(cv2.cvtColor(lum_image, cv2.COLOR_BGR2RGB)), plt.title('Luminescence Image')
plt.subplot(143), plt.imshow(cv2.cvtColor(lum_image_with_contours, cv2.COLOR_BGR2RGB)), plt.title('Contours on Luminescence Image')
plt.subplot(144), plt.imshow(cv2.cvtColor(original_image_with_contours, cv2.COLOR_BGR2RGB)), plt.title('Contours on Original Image')

# Ensure the plots are shown properly
plt.tight_layout()
plt.show()